In [ ]:
import requests
import findspark
import json

#initialize spark installation
findspark.init()

from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession

# The Movie DB api key and endpoint
key='233da043e658c47fd8661a48c2287fce'
url = 'https://api.themoviedb.org/3/'
        

In [ ]:
if __name__ == '__main__':
    # Create new spark session and context
    my_spark = SparkSession.builder.appName("movieData").master("local").getOrCreate() 
    sc = my_spark.sparkContext
    
    # Request now playing movies
    now_playing = '{}movie/{}?api_key={}'.format(url,"now_playing",key)
    response = requests.get(now_playing)
    # format response
    out = response.json()
    out = [json.dumps(out)]
    
    # Read data into spark
    rdd = sc.parallelize(out)
    data = my_spark.read.json(rdd)


In [ ]:
from pymongo import MongoClient

# Connect to Mongo and Movie database
client = MongoClient('localhost', 27017) 
db = client.movieDatabase 
movies = db.movies

# Create list with all movie id
id_list = []
for ids in movies.find({},{ "title": 0, "title_low":0, "title_hashtag":0 }):
    id_list.append(list(ids.values())[0])
    
pdData = data.toPandas()
results = pdData['results'][0]
new_id_list = []

# Foreach movie add/update it to the mongo collection
for v,w in enumerate(results):
    # Only add movie if it is an englisch movie
    if results[v]['original_language']=='en':
        # Transform the data
        title = results[v]['title']
        title_low = title.lower()
        title_hashtag = ''.join(('#',title_low.replace(" ", "")))
        movie= {"_id": results[v]['id'], "title": title, "title_low":title_low, "title_hashtag":title_hashtag}
        movies.update_one({ "_id" : results[v]['id']},
                { "$set": { "title": title, "title_low":title_low, "title_hashtag":title_hashtag }}, upsert=True
            )
        new_id_list.append(results[v]['id'])

# Remove movies that are no longer playing from the collection
movies.delete_many( {'_id': { '$in': list( set(id_list) - set(new_id_list) ) } } )
        
